In [4]:
import cv2
import math, sys
import matplotlib.pyplot as plt

'''
savefig sould be done before show
see: https://blog.csdn.net/u010099080/article/details/52912439
'''
def img_hist(img_in):
    hist = []
    for i in range(256):
        hist.append(0)
    
    row, col= img_in.shape
    for i in range(0, row):
        for j in range(0, col):
            hist[img_in[i, j]] += 1
    
    plt.bar(range(0, 256), hist)
    plt.savefig('histogram.png') 
    plt.show()
    return 0

def img_binarize(img_in):
    return (img_in > 0x7f) * 0xff

def connected_components(img_in):
    disjoint_set =[[]]
    
    img_padded = cv2.copyMakeBorder(img_in, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value = 0) # padding img with 0 
    # 1st checking
    img_padded = (img_padded > 0x7f) * 1
    plt.imshow(img_padded, cmap = 'gray')
    row, col = img_padded.shape
    
    # 1st checking
    labeling_index = 1
    for i in range(0, row):
        for j in range(0, col):
            # check left
            
            # check up
            
            # check self
            
             
    return 0

img = cv2.imread('lena.bmp', 0)
img_binarized = img_binarize(img)
plt.imshow(img_binarized, cmap = 'gray')
plt.savefig('lena_binarized.png', cmap = 'gray')
plt.show()

plt.clf()
img_hist(img)

connected_components(img_binarized)

IndentationError: expected an indented block (<ipython-input-4-2ea2137c05da>, line 43)